<a href="https://colab.research.google.com/github/DerNiccoo/TheEarlyBird/blob/main/Aufgab4/Alina/Assignment4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Assignment 4**

**1. Go to the following Git repo which contains a dataset of German articles:
https://github.com/tblock/10kGNAD**<br>
**2. Download the file train.csv from there and import it into a notebook.**

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import torch

In [ ]:
import pandas as pd

#Read the CSV file with seperator= ; and set Columnnames 'labels' and 'text'
df = pd.read_csv('/content/drive/My Drive/KI_Lab/train.csv', error_bad_lines=False, sep=';', names=['labels','text'])

In [ ]:
print(df)

             labels                                               text
0             Sport  21-Jähriger fällt wohl bis Saisonende aus. Wie...
1            Kultur  'Erfundene Bilder zu Filmen, die als verloren ...
2               Web  Der frischgekürte CEO Sundar Pichai setzt auf ...
3        Wirtschaft  Putin: "Einigung, dass wir Menge auf Niveau vo...
4            Inland  Estland sieht den künftigen österreichischen P...
...             ...                                                ...
9240         Inland  Bernd Saurer war Bridge-Juniorenweltmeister un...
9241  International  Sandhere soll in vergangener Woche bei Luftang...
9242     Wirtschaft  Derzeit Konzeptgruppe in Berlin – Kein Komment...
9243         Inland  Landeshauptmann will den vierten Regierungssit...
9244       Panorama  Er ist einer von Millionen syrischen Flüchtlin...

[9245 rows x 2 columns]


In [ ]:
#From github
#url = 'https://raw.githubusercontent.com/tblock/10kGNAD/master/train.csv'
#df = pd.read_csv(url, error_bad_lines=False)

**3. Drop the label and only use the sentence in each line (the part after the first ; ).**

In [ ]:
df = df.drop('labels', axis = 1)

**Install spacy on GPU**


In [ ]:
!pip install -U spacy[cuda100] de


     |████████████████████████████████| 10.4MB 5.7MB/s 
     |████████████████████████████████| 1.1MB 38.7MB/s 
     |████████████████████████████████| 348.0MB 45kB/s 
  Created wheel for de: filename=de-0.1-cp36-none-any.whl size=1158 sha256=0008ff32939651e9d010e3e87a8e6305a144dac22a26eddc3adce6cb63e5d82a
  Stored in directory: /root/.cache/pip/wheels/fd/16/b9/46fac6c03128929538323f5457e947af752b7c58d4c7783f04
Successfully built de
  Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [ ]:
!python -m spacy download de
import spacy 

     |████████████████████████████████| 14.9MB 688kB/s 
  Created wheel for de-core-news-sm: filename=de_core_news_sm-2.3.0-cp36-none-any.whl size=14907580 sha256=b3d72fd51801093236bdecb78a73f251ce6102c74733085128bf7a8a8b63ce07
  Stored in directory: /tmp/pip-ephem-wheel-cache-2au8vnh6/wheels/db/f3/1e/0df0f27eee12bd1aaa94bcfef11b01eca62f90b9b9a0ce08fd
Successfully built de-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/de_core_news_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/de
You can now load the model via spacy.load('de')


In [ ]:
#Spacy on GPU
gpu = spacy.prefer_gpu()
print('GPU:', gpu)

GPU: True


**4. Use spaCy to get all nouns in the whole file.**

In [ ]:
nlp = spacy.load("de")

In [ ]:
#Safe all nouns from Column 'text' in list_nouns
list_nouns =[]

for index, row in df.iterrows():
  doc = nlp(row['text'])
  for token in doc:
    if token.pos_ == 'NOUN': 
      list_nouns.append(token.text)

In [ ]:
#print(list_nouns)

**5. Filter out the nouns that have characters wich are not in all_letters (see
code snippet below). The remaining set of nouns constitute our training data.**

In [ ]:
import string
all_letters = string.ascii_letters + "ÄÖÜäöüß"
list_nouns_ascii = []

for noun in list_nouns:
  contained = True
  
  for char in noun:
    if char not in all_letters:
      contained = False
      break

  if contained:
    list_nouns_ascii.append(noun)

#print(list_nouns_ascii)

**6. Go through the PyTorch tutorial on name generation. Clone the notebook, execute it
and understand what is going on.**<br>
**7. Use the same architecture to train a language model on the nouns that were extracted
from the German dataset in step 1 to 5. (Note: We do not need a „category“, you have
to remove everything that belongs to „category“ from the code).**

In [ ]:
#Connect to GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [ ]:
#Define for Training and RNN needed variables (to not have to delete everything that belongs to category and reuse the RNN e.g. with verbs) 
n_letters = len(all_letters) + 1 # Plus EOS marker

#Only need one categorie = all nouns from list_nouns
n_categories = 1

#Dictionary for nouns
category_lines = {'NOUN': list_nouns_ascii}
all_categories = ['NOUN']

**RNN Class definiton**

In [ ]:
import torch
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.hidden_size = hidden_size

        self.i2h = nn.Linear(n_categories + input_size + hidden_size, hidden_size)
        self.i2o = nn.Linear(n_categories + input_size + hidden_size, output_size)
        self.o2o = nn.Linear(hidden_size + output_size, output_size)
        self.dropout = nn.Dropout(0.1)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, category, input, hidden):
        input_combined = torch.cat((category, input, hidden), 1)
        hidden = self.i2h(input_combined)
        output = self.i2o(input_combined)
        output_combined = torch.cat((hidden, output), 1)
        output = self.o2o(output_combined)
        output = self.dropout(output)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [ ]:
import random

# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

# Get a random category and random line from that category
def randomTrainingPair():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    return category, line

In [ ]:
# One-hot vector for category
def categoryTensor(category):
    li = all_categories.index(category)
    tensor = torch.zeros(1, n_categories)
    tensor[0][li] = 1
    return tensor

# One-hot matrix of first to last letters (not including EOS) for input
def inputTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li in range(len(line)):
        letter = line[li]
        tensor[li][0][all_letters.find(letter)] = 1
    return tensor

# LongTensor of second letter to end (EOS) for target
def targetTensor(line):
    letter_indexes = [all_letters.find(line[li]) for li in range(1, len(line))]
    letter_indexes.append(n_letters - 1) # EOS
    return torch.LongTensor(letter_indexes)

In [ ]:
# Make category, input, and target tensors from a random category, line pair
def randomTrainingExample():
    category, line = randomTrainingPair()
    category_tensor = categoryTensor(category)
    input_line_tensor = inputTensor(line)
    target_line_tensor = targetTensor(line)
    return category_tensor.to(device), input_line_tensor.to(device), target_line_tensor.to(device)

In [ ]:
criterion = nn.NLLLoss()

learning_rate = 0.0005

def train(category_tensor, input_line_tensor, target_line_tensor):
    target_line_tensor.unsqueeze_(-1)
    hidden = rnn.initHidden()

    rnn.zero_grad()

    loss = 0

    for i in range(input_line_tensor.size(0)):
        output, hidden = rnn(category_tensor, input_line_tensor[i], hidden)
        l = criterion(output, target_line_tensor[i])
        loss += l

    loss.backward()

    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)

    return output, loss.item() / input_line_tensor.size(0)

In [ ]:
import time
import math

def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)

**Training of RNN**

In [ ]:
rnn = RNN(n_letters, 128, n_letters)

n_iters = 100000
print_every = 5000
plot_every = 500
all_losses = []
total_loss = 0 # Reset every plot_every iters

start = time.time()

for iter in range(1, n_iters + 1):
    output, loss = train(*randomTrainingExample())
    total_loss += loss

    if iter % print_every == 0:
        print('%s (%d %d%%) %.4f' % (timeSince(start), iter, iter / n_iters * 100, loss))

    if iter % plot_every == 0:
        all_losses.append(total_loss / plot_every)
        total_loss = 0

0m 59s (5000 5%) 2.6567
1m 57s (10000 10%) 2.4289
2m 56s (15000 15%) 2.5321
3m 55s (20000 20%) 3.3504
4m 52s (25000 25%) 3.1636
5m 52s (30000 30%) 1.8621
6m 51s (35000 35%) 2.9568
7m 50s (40000 40%) 2.1840
8m 49s (45000 45%) 3.3009
9m 47s (50000 50%) 1.7498
10m 46s (55000 55%) 2.2324
11m 44s (60000 60%) 2.3208
12m 43s (65000 65%) 1.9596
13m 41s (70000 70%) 1.9722
14m 40s (75000 75%) 2.5619
15m 40s (80000 80%) 3.3646
16m 39s (85000 85%) 2.6043
17m 37s (90000 90%) 1.9493
18m 37s (95000 95%) 3.1950
19m 34s (100000 100%) 1.2906


**Sampling-Methods**

Original Sampling-Method

In [ ]:
max_length = 20

# Sample from a category and starting letter
def sample(category, start_letter='A'):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letter)
        hidden = rnn.initHidden()

        output_name = start_letter


        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name

# Get multiple samples from one category and multiple starting letters
def samples(category, start_letters='ABC'):
    for start_letter in start_letters:
        print(sample(category, start_letter))

samples('NOUN', 'HLCG')


Hien
Land
Catstarite
Geschaft


**8. Extend the sampling logic in the sample function sucht that:
<br>
i. The model extends any given prefix string by first feeding the prefix into the RNN
and then start generating new characters. (Currently the model only samples based
on a given starting letter).**

In [ ]:
# Sample from a category and whole prefix
def sample_highest_score(category, start_letters):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letters)
        hidden = rnn.initHidden()

        output_name = start_letters

        #Loop over the Input Letters except the last one and create input tensors for each
        for character in start_letters[:-1]:
          input = inputTensor(character)

          # Calculate the output and current hidden layer for each input tensor with defined RNN, 
          # to learn the models hidden layers with first given letters 
          output, hidden = rnn(category_tensor, input[0], hidden)

        #Create Input Tensor for last character and overwrite current Input Tensor
        input = inputTensor(start_letters[-1])

        #Calculate the following predicted letters after the last Input character based on learned RNN/ hidden_layers    
        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)

            #Take the character with the highest score 
            topv, topi = output.topk(1)
            topi = topi[0][0]
            if topi == n_letters - 1:
                break
            else:
                letter = all_letters[topi]
                output_name += letter
            input = inputTensor(letter)

        return output_name


**ii. Currently the model uses the highest score during sampling. Change that into
drawing a real sample from the softmax distribution.**

In [ ]:
import numpy as np

#Convert all_letters to list and add <EOS> 
all_letters_with_eos = []
for character in all_letters:
  all_letters_with_eos.append(character)
all_letters_with_eos.append("<EOS>")

# Sample from a category and starting letter
def sample_real_sampling(category, start_letters):
    with torch.no_grad():  # no need to track history in sampling
        category_tensor = categoryTensor(category)
        input = inputTensor(start_letters)
        hidden = rnn.initHidden()

        output_name = start_letters

        #Loop over the Input Letters except the last one and create input tensors for each
        for character in start_letters[:-1]:
          input = inputTensor(character)

          # Calculate the output and current hidden layer for each input tensor with defined RNN, 
          # to learn the models hidden layers with first given letters 
          output, hidden = rnn(category_tensor, input[0], hidden)

        #Create Input Tensor for last character and overwrite current Input Tensor
        input = inputTensor(start_letters[-1])

        for i in range(max_length):
            output, hidden = rnn(category_tensor, input[0], hidden)

            # Cast output Tensor to numpy array with dimension 1 and cast it with exp() to a normal SoftMax distribution
            output = output.cpu().detach().numpy().ravel()
            output = np.exp(output)
           
            #Do a random select with the probability_distribution of all letters in all_letters_with_eos
            top_char = random.choices(all_letters_with_eos, output, k=1)

            if top_char[0] == "<EOS>":
                break
            else:
                output_name += top_char[0]
            input = inputTensor(top_char)

        return output_name


**9. Choose 10 random words from the training data and randomly cut-off a prefix. For
every of those prefixes give an output for:**
> **1. The model extending the prefix with the max score approach.**<br>
> **2. The mode extending the prefix with a real sampling approach.**<br>
> **3. Randomly pick characters from all_letters to extend the prefix (this is just for comparison reasons, i.e. how would a complete random generation look like).**




In [ ]:
# Random item from a list
def randomChoice(l):
    return l[random.randint(0, len(l) - 1)]

In [ ]:
#Choose 10 random words from list_nouns_ascii
for i in range(10):
  word = randomChoice(list_nouns_ascii)
  prefix = word[0:random.randint(1, len(word))]

  #Complete random generated string for comparison
  string = "".join((random.choices(all_letters, k=random.randint(2,6))))
  rand_word = prefix + string

  original = sample('NOUN', prefix)
  highest_score = sample_highest_score('NOUN', prefix)
  weighted_sampling = sample_real_sampling('NOUN', prefix)

  
  print('Word: {:30} Prefix: {:30} OriginalSampling: {:30} HighestScore: {:30} WeightedSampling: {:30} RandomWordGeneration: {:30}'. format(word, prefix, original, highest_score, weighted_sampling, rand_word))



Word: Nutzerdaten                    Prefix: Nutzerdat                      OriginalSampling: Nutzerdataten                  HighestScore: Nutzerdat                      WeightedSampling: Nutzerdat                      RandomWordGeneration: NutzerdatRRäkAE               
Word: Stadtregierung                 Prefix: Stadtre                        OriginalSampling: Stadtretanden                  HighestScore: Stadtren                       WeightedSampling: StadtrebusO                    RandomWordGeneration: StadtrerIäYCc                 
Word: Sohn                           Prefix: So                             OriginalSampling: Sotindert                      HighestScore: Sondare                        WeightedSampling: SodlÖxsterkeispe               RandomWordGeneration: SorcZ                         
Word: Zeichen                        Prefix: Zeiche                         OriginalSampling: Zeicheeit                      HighestScore: Zeichen                        WeightedSampli